# Script generalizado - Calçadas vs Viagens de Pedestres

In [ ]:
# Constantes
pedestrians_od_value = 17 # o que está na pesquisa OD 2023

# exemplo de Zona OD
zone_name = "Fazenda Itaim" 
zone_number = 205 
save_map_as = "faz_itaim" 

pedestrians_buffer_value = 15
sidewalks_buffer_value = 5

In [ ]:
# imports
import pandas as pd
import geopandas as gpd
import folium
from shapely.ops import transform
import pyproj

In [ ]:
# função auxiliar
def swap_xy(geom):
    """
    Inverte as coordenadas X e Y de uma geometria: (lat, lon) → (lon, lat)

    Args:
        geom (shapely.geometry.base.BaseGeometry): Objeto de geometria Shapely
            cuja ordem das coordenadas será invertida.

    Returns:
        shapely.geometry.base.BaseGeometry: Nova geometria com as coordenadas X e Y invertidas.
    """
    return transform(lambda x, y: (y, x), geom)

Carregar datasets

In [ ]:
od23_path = "../../../data/OD/ODS/od23_all.csv"
od = pd.read_csv(od23_path)

od_zones_path = "../../../data/OD/OD-2023/002_Site Metro Mapas_190225/Shape/Zonas_2023.shp"
od_zones = gpd.read_file(od_zones_path).to_crs("EPSG:4326")

sidewalks_path = "../../../data/Geosampa/SIRGAS_SHP_calcada/SIRGAS_SHP_calcada.shp"
sidewalks = gpd.read_file(sidewalks_path)
sidewalks = sidewalks.set_crs("EPSG:31983").to_crs("EPSG:4326")

### Filtragem da área Zona OD e pedestres da Zona OD

Filtrar zona OD

In [ ]:
# Filtrar zona da Zona OD
analysis_zone = od_zones.query("NomeZona == @zone_name").copy() 
analysis_zone

Filtrar pedestres e origens da zona OD 

In [ ]:
# Filtrar pedestres e origens da Zona OD
zone_pedestrians = od.query("MODOPRIN == @pedestrians_od_value and ZONA_O == @zone_number")

# Criar coluna de geometria a partir de coordenadas de origem da viagem a pé
pedestrians_gdf = gpd.GeoDataFrame(
    zone_pedestrians,
    geometry=gpd.points_from_xy(
        zone_pedestrians["CO_O_X"], zone_pedestrians["CO_O_Y"]
    ),
    crs="EPSG:4326",
)

Bufferiza a geometria dos pontos de origem dos pedestres da zona OD

In [ ]:
# Expande um buffer de 30m para representar área de influência da origem
pedestrians_gdf = pedestrians_gdf.to_crs("EPSG:31983") # CRS para operações geoespaciais
pedestrians_gdf["geometry"] = pedestrians_gdf["geometry"].buffer(pedestrians_buffer_value)

pedestrians_gdf = pedestrians_gdf.to_crs("EPSG:4326") # CRS para operações com folium (mapas)

pedestrians_gdf["geometry"] = pedestrians_gdf["geometry"].apply(swap_xy)

### Intersecção entre calçadas da Zona OD e Viagens de Pedestres

Preparar calçadas e aplicar clip('intersection') na Zona OD

- Bufferiza a geometria das calçadas 
- Renomeia coluna index

In [ ]:
sidewalks = sidewalks.to_crs("EPSG:31983") # para operações geoespaciais como buffer
sidewalks["geometry"] = sidewalks["geometry"].buffer(sidewalks_buffer_value)
sidewalks = sidewalks.to_crs("EPSG:4326")

# clip(intesection) de calçadas apenas dentro da zona
sidewalks_analysis_zone = gpd.clip(sidewalks, analysis_zone).reset_index() 
sidewalks_analysis_zone = sidewalks_analysis_zone.rename(columns={"index": "index_sidewalk"})

print(f"Nº de calçadas na '{zone_name}': {len(sidewalks_analysis_zone)}")

Intersecção espacial entre as tabelas de pedestres e calçadas

In [ ]:
intersections = gpd.sjoin(
    pedestrians_gdf,
    sidewalks_analysis_zone,
    how="inner",
    predicate="intersects",
)

print(f"Intersecções de ponto de origem e calçadas: {len(intersections)}")

### Cálculo da intensidade das viagens a pé nas calçadas da Zona OD

Agrupar intensidade (FE_VIA) por calçada

In [ ]:
sidewalks_usage = (
    intersections.groupby("index_sidewalk")["FE_VIA"]
    .sum()
    .reset_index()
    .rename(columns={"FE_VIA": "INTENSIDADE"})
)

Merge com camada de calçadas

In [ ]:
sidewalk_ped_plot = sidewalks_analysis_zone.merge( 
    sidewalks_usage, on="index_sidewalk", how="left"
)
sidewalk_ped_plot["INTENSIDADE"] = sidewalk_ped_plot[
    "INTENSIDADE"
].fillna(0)

Filtrar apenas calçadas com intensidade(fe_via) > 0

In [ ]:
sidewalk_ped_plot = sidewalk_ped_plot.query("INTENSIDADE > 0")

print(f"Calçadas com uso registrado: {len(sidewalk_ped_plot)}")

### Cálcula representatividade de viagens a pé das calçadas da Zona OD

Totais de viagens e total de viagens de pedestres da Zona OD

In [ ]:
zone_all_means_sum = od.query("ZONA_O == @zone_number")["FE_VIA"].sum() 
zone_walk_sum = od.query("ZONA_O == @zone_number and MODOPRIN == @pedestrians_od_value")["FE_VIA"].sum() 

Criar colunas para porcentagens de totais: viagens e viagens a pé da Zona OD

In [ ]:
sidewalk_ped_plot["PERC_DIST"] = 100 * sidewalk_ped_plot["INTENSIDADE"] / zone_all_means_sum
sidewalk_ped_plot["PERC_PED"] = 100 * sidewalk_ped_plot["INTENSIDADE"] / zone_walk_sum

## Mapa: Calçadas vs Viagens Pedestres

Centro do mapa - zona OD em foco

In [ ]:
center_proj = analysis_zone.to_crs(31983)
map_center = center_proj.geometry.centroid.iloc[0] # centro
map_center = gpd.GeoSeries([map_center], crs=31983).to_crs(4326).iloc[0]
map_center = [map_center.y, map_center.x]

# declaração do mapa
sidewalk_ped_map = folium.Map(location=map_center, zoom_start=15, tiles="cartodb positron")

mapa coroplético das calçadas em razão de FE_VIA das viagens a pé

In [ ]:
# camada coroplética
folium.Choropleth(
    geo_data=sidewalk_ped_plot.to_json(),
    data=sidewalk_ped_plot,
    columns=["index_sidewalk", "INTENSIDADE"],
    key_on="feature.properties.index_sidewalk",
    fill_color="YlOrRd",
    fill_opacity=0.7,
    line_opacity=0.3,
    legend_name="Intensidade de viagens (FE_VIA)",
).add_to(sidewalk_ped_map)

popup com logradouro(rua, avenida...) e representativdade de viagens a pé da calçada

In [ ]:
# camada interativa com popup
for _, row in sidewalk_ped_plot.iterrows():
    popup_html = (
        f"<b>Logradouro:</b> {row['cc_lognome']}<br>"
        f"<b>Intensidade (Fator de expansão):</b> {row['INTENSIDADE']}<br>"
        f"<b>% sobre total da zona OD:</b> {row['PERC_DIST']:.2f}%<br>"
        f"<b>% sobre pedestres da zona OD:</b> {row['PERC_PED']:.2f}%"
    )
    folium.GeoJson(
        row["geometry"],
        popup=folium.Popup(popup_html, max_width=300),
        style_function=lambda x: {"color": "black", "weight": 1, "fillOpacity": 0},
    ).add_to(sidewalk_ped_map)

Salvar mapa 'Calçadas vs Pedestres' da zona OD no diretório [built_maps](../../built_maps/)

In [ ]:
try:
    save_map_dir_path = "../built_maps/sidewalks_maps/" + save_map_as +"_calcadas_pedestres_gen_script.html"
    sidewalk_ped_map.save(save_map_dir_path)
    print(f"Mapa salvo: {save_map_dir_path}")
except Exception as e:
    print(f"Erro ao salvar o mapa: {e}")